## Kafka

In [ ]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer

### Admin

In [ ]:
broker = "localhost:9092"
admin = KafkaAdminClient(bootstrap_servers=[broker])

In [ ]:
# List topics
admin.list_topics()

### Creating topics

In [ ]:
from kafka.admin import NewTopic

In [ ]:
# Create new topic
NewTopic(name="even_nums", num_partitions=1, replication_factor=1)

In [ ]:
# Create topic for admin
admin.create_topics([NewTopic(name="even_nums", num_partitions=1, replication_factor=1)])

In [ ]:
# Delete topic for admin
admin.delete_topics(["even_nums"])

### Topics for even numbers and odd numbers

In [ ]:
from kafka.errors import TopicAlreadyExistsError

In [ ]:
# Create even_nums topic
try:
    admin.create_topics([NewTopic(name="even_nums", num_partitions=1, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [ ]:
# Create odd_nums topic
try:
    admin.create_topics([NewTopic(name="odd_nums", num_partitions=2, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [ ]:
admin.list_topics()

**Note:** Kafka creates a topic if you start writing messages to a non-existing topic. This might be undesirable for your application. We have taken care of this already by including `auto.create.topics.enable=false` configuration insider our `Dockerfile`.

### Producer

In [ ]:
producer = KafkaProducer(bootstrap_servers=[broker])

In [ ]:
# Send message
result = producer.send("even_nums", bytes(str(0), "utf-8"))
result

In [ ]:
result.get()

In [ ]:
from threading import Thread, Lock

lock = Lock()
def Print(*args):
    with lock:
        print(*args)

Print("hi")

### Producer code for odd_nums and even_nums

In [ ]:
import time, threading

In [ ]:
def num_producer(topic, start, step):
    producer = KafkaProducer(bootstrap_servers=[broker])
    num = start
    while True:
        if num < 10:
            Print("Send", num, "to: ", topic)
        producer.send(topic, bytes(str(num), "utf-8"))
        num += step
        time.sleep(1)

threading.Thread(target=num_producer, args=("even_nums", 0, 2)).start()
threading.Thread(target=num_producer, args=("odd_nums", 1, 2)).start()

### Consumer

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=[broker])

In [ ]:
# Polling
batch = consumer.poll(1000)
batch

In [ ]:
# Assignment
consumer.assignment()

### Manual Partition Assignment

In [ ]:
from kafka import TopicPartition

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.assign([TopicPartition("even_nums", 0)])
consumer.assignment()

In [ ]:
batch = consumer.poll(1000)
batch

### Starting consumption from the beginning

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.assign([TopicPartition("even_nums", 0)])
consumer.seek_to_beginning()
consumer.assignment()

In [ ]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

### Automatic Partition Assignment

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums"])
print(consumer.assignment())

In [ ]:
_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

In [ ]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

### Multiple Assignment

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums", "odd_nums"])
print(consumer.assignment())

_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

In [ ]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    print(topic_partition)
    for msg in messages:
        print(int(str(msg.value, "utf-8")))